In [12]:
import numpy as np
import matplotlib.pyplot as plt
from getMatrix import getMatrix
from randomSVD import randomSVD
from randomColumnSelection import randomColumnSelection
from helpers import coherence

In [13]:
n = 96
k = 48
res = 1e-12
trialNumber = 20
normalization = 1e-12
pRange = range(0,9,2)

maxPower = 4
sketchType = 'Gaussian'

In [15]:
svdData = {}

for p in pRange:
    
    coherences = []
    errors = []
    qs = []

    for coherenceScalar in np.arange(0,.3,.001)[::-1]:

        trialErrors = []

        #Generate A and calculate coherence
        U, sigma, V = getMatrix(n,k,res,'coherent','smooth gap',True, coherenceScalar)
        A = U@sigma@V.T
        for q in range(maxPower):
            coherences.append(coherence(V,k))

            #Generate Â and calculate approximation error
            for j in range(trialNumber):
                Ahat= randomSVD(A,k,p,q,sketchType)
                trialErrors.append(np.linalg.norm(Ahat-A, ord = 2)/normalization)
            errors.append(np.average(np.array(trialErrors)))
            qs.append(q)

        trialErrors = []

        #Generate A and calculate coherence
        U, sigma, V = getMatrix(n,k,res,'incoherent','smooth gap',True, coherenceScalar)
        A = U@sigma@V.T
        for q in range(maxPower):
            coherences.append(coherence(V,k))

            #Generate Â and calculate approximation error
            for j in range(trialNumber):
                Ahat= randomSVD(A,k,p,q,sketchType)
                trialErrors.append(np.linalg.norm(Ahat-A, ord = 2)/normalization)
            errors.append(np.average(np.array(trialErrors)))
            qs.append(q)

    #Sort by coherence for nice graphing
    sortingIndices = np.argsort(coherences)
    coherences = np.array(coherences)[sortingIndices]
    errors = np.array(errors)[sortingIndices]
    qs = np.array(qs)[sortingIndices]
    svdData[p]=[coherences,qs,errors]

In [29]:
#Graphing
%matplotlib qt
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.set_zscale('log')

ax.set_xlabel('Coherence')
ax.set_ylabel('Power')
ax.set_zlabel('Approximation Error')
ax.set_yticks(range(maxPower))

for p in svdData.keys():
    coherences, qs, errors = svdData[p]
    ax.scatter3D(coherences,qs,errors)